## Import Library
Pada tahap ini, kita akan mengimpor library yang diperlukan untuk analisis data dan visualisasi. Beberapa library yang akan digunakan di antaranya:

In [1]:
# Import library yang akan digunakan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import requests
import folium

import warnings
warnings.filterwarnings('ignore')

## Load Dataset
Langkah pertama dalam analisis ini adalah memuat dataset yang akan digunakan. 
Impor data dilakukan dari dua sumber: 
- Dataset utama dalam format CSV yang berisi informasi mengenai listing Airbnb di Bangkok.
- Data GeoJSON yang menggambarkan distrik-distrik di Bangkok yang akan digunakan untuk visualisasi geografis.

In [2]:
# Load clean dataset Airbnb Listings Bangkok
url = 'https://raw.githubusercontent.com/Quinntes/airbnb_listings_bangkok/refs/heads/main/cleaned_airbnb_bangkok.csv'
df = pd.read_csv(url)

In [5]:
df.head()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,has_review
0,27934,Nice room with superb city view,120437,Nuttee,Ratchathewi,13.75983,100.54134,Entire home/apt,1905.0,3,65,2020-01-06,0.50,2,353,0,1
1,27979,"Easy going landlord,easy place",120541,Emy,Bang Na,13.66818,100.61674,Private room,1316.0,1,0,NaN,0.00,2,358,0,0
2,28745,modern-style apartment in Bangkok,123784,Familyroom,Bang Kapi,13.75232,100.62402,Private room,800.0,60,0,NaN,0.00,1,365,0,0
3,35780,Spacious one bedroom at The Kris Condo Bldg. 3,153730,Sirilak,Din Daeng,13.78823,100.57256,Private room,1286.0,7,2,2022-04-01,0.03,1,323,1,1
4,941865,Suite Room 3 at MetroPoint,610315,Kasem,Bang Kapi,13.76872,100.63338,Private room,1905.0,1,0,NaN,0.00,3,365,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15838 entries, 0 to 15837
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              15838 non-null  int64  
 1   name                            15838 non-null  object 
 2   host_id                         15838 non-null  int64  
 3   host_name                       15838 non-null  object 
 4   neighbourhood                   15838 non-null  object 
 5   latitude                        15838 non-null  float64
 6   longitude                       15838 non-null  float64
 7   room_type                       15838 non-null  object 
 8   price                           15838 non-null  float64
 9   minimum_nights                  15838 non-null  int64  
 10  number_of_reviews               15838 non-null  int64  
 11  last_review                     10057 non-null  object 
 12  reviews_per_month               

In [7]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,has_review
count,1.583800e+04,1.583800e+04,15838.000000,15838.000000,1.583800e+04,15838.000000,15838.000000,15838.000000,15838.000000,15838.000000,15838.000000,15838.000000
mean,1.580992e+17,1.541276e+08,13.745147,100.559897,3.202245e+03,14.603738,16.645347,0.516450,13.900871,244.387865,3.484720,0.634992
std,2.947075e+17,1.318700e+08,0.043038,0.050917,2.494666e+04,44.715992,40.569606,0.952981,30.282910,125.812240,8.920788,0.481448
min,2.793400e+04,5.892000e+04,13.527300,100.329550,2.780000e+02,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,2.104510e+07,3.974431e+07,13.720092,100.529690,9.000000e+02,1.000000,0.000000,0.000000,1.000000,138.000000,0.000000,0.000000
50%,3.503734e+07,1.224556e+08,13.738490,100.561400,1.429000e+03,1.000000,2.000000,0.090000,4.000000,309.000000,0.000000,1.000000
75%,5.259990e+07,2.390547e+08,13.759497,100.585148,2.429000e+03,7.000000,13.000000,0.670000,13.000000,360.000000,3.000000,1.000000
max,7.908162e+17,4.926659e+08,13.953540,100.923440,1.100000e+06,365.000000,1224.000000,19.130000,228.000000,365.000000,325.000000,1.000000


# Exploratory Data Analysis (EDA)
Setelah data bersih dan kaya fitur, maka selanjutnya masuk ke tahap *Exploratory Data Analysis (EDA)*. Di sini, tiap visualisasi dan analisis akan diarahkan untuk menjawab pertanyaan bisnis yang sudah dirumuskan.

## 1. Distribusi Dasar dari Atribut Utama dalam Dataset
Sebelum masuk ke faktor performa seperti harga, review, atau okupansi, kita perlu memahami dulu landscape supply alias karakteristik penawaran listing di Airbnb Bangkok.

### 1.1 `room_type` Apa yang Paling Dominan di Bangkok?
Untuk memahami dinamika pasar listing Airbnb di Bangkok, pertama-tama perlu mengetahui tipe kamar yang paling dominan. Hal ini penting karena tipe kamar menentukan ekspektasi tamu serta pengelolaan dan harga yang diterapkan oleh host.

In [ ]:
# palet Rausch
rausch = ['#FF5A5F', '#FF7A7E', '#FF9A9F', '#FFBBBF']

plt.figure(figsize=(8, 5))
sns.countplot(
    data=df,
    x='room_type',
    order=df['room_type'].value_counts().index,
    palette=rausch
)
plt.title('Room Type Paling Dominan di Airbnb Bangkok')
plt.xlabel('Room Type', fontsize=12)
plt.ylabel('Jumlah Listing', fontsize=12)
plt.show()

**Insight :**
- Entire home/apt terlihat jauh lebih tinggi jumlah listingnya dibandingkan 3 tipe lainnya.
- Berdasarkan praktik umum di pasar Airbnb, sewa satu unit penuh (Entire home/apt) adalah jenis paling populer secara global, karena menawarkan:
    - Privasi penuh
    - Fleksibilitas waktu
    - Cocok untuk keluarga atau grup
- Shared room dan Hotel room cenderung:
    - Dipilih oleh budget travelers.
    - Kurang populer jika dibandingkan dengan privasi yang ditawarkan oleh unit full.

Setelah mengetahui tipe kamar yang paling banyak ditemukan, selanjutnya melihat di mana lokasi atau neighborhood dengan jumlah listing terbanyak, untuk memahami kawasan mana yang paling kompetitif di pasar.

### 1.2 `Neigborhood` Mana yang Memiliki Supply Listing Terbanyak?
Mengetahui lokasi dengan jumlah listing terbanyak akan memberi wawasan tentang area yang paling kompetitif di pasar Airbnb Bangkok. Beberapa neighbourhood mungkin memiliki konsentrasi supply yang tinggi, yang bisa mengindikasikan bahwa di area tersebut, persaingan antara host lebih ketat.

Berikut adalah peta distribusi listing Airbnb di Bangkok, yang memberikan gambaran tentang konsentrasi listing di berbagai neighborhood. Peta ini akan membantu kita untuk lebih mudah memahami di mana lokasi yang memiliki jumlah listing terbanyak dan akan menjadi titik fokus analisis berikutnya.

In [ ]:
import folium
from folium.plugins import MarkerCluster

# Membuat peta dasar
map_bangkok = folium.Map(location=[13.7563, 100.5018], zoom_start=12)

# Menambahkan MarkerCluster untuk menandai lokasi listing
marker_cluster = MarkerCluster().add_to(map_bangkok)

# Menambahkan marker untuk setiap listing
for idx, row in df_copy.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['neighbourhood']
    ).add_to(marker_cluster)

# Menampilkan peta
map_bangkok.save('bangkok_airbnb_map.html')
map_bangkok

In [ ]:
# Memeriksa 10 neighbourhood dengan jumlah listing terbanyak
top_neigh = df_copy['neighbourhood'].value_counts().head(10)

# Visualisasi menggunakan barplot
plt.figure(figsize=(10, 5))
sns.barplot(x=top_neigh.values, y=top_neigh.index, color='#FF5A5F')
plt.title('10 Neighborhood dengan Jumlah Listing Tertinggi')
plt.xlabel('Jumlah Listing')
plt.ylabel('Neighbourhood')
plt.show()

**Insight :**
- Vadhana dan Khlong Toei menempati posisi teratas, masing-masing dengan lebih dari 2.000 listing.
- Tingginya jumlah listing di beberapa neighborhood dapat mengindikasikan:
    - Konsentrasi supply yang lebih tinggi.
    - Potensi kompetisi antar host yang lebih ketat di area tersebut.

Namun, mengetahui hanya jumlah listing belum cukup untuk memberi gambaran menyeluruh. Oleh karena itu, perlu mengkaji lebih dalam apakah ada keterkaitan antara tipe kamar yang dominan dengan lokasi tertentu. Dengan kata lain, tipe kamar tertentu mungkin lebih populer di neighborhood tertentu.

### 1.3 Apakah tipe listing `room_type` tertentu lebih umum di area-area tertentu?
Melalui pemetaan tipe kamar berdasarkan neighborhood, kita dapat melihat apakah preferensi pasar di suatu daerah berhubungan dengan jenis kamar yang ditawarkan. Hal ini dapat memberikan wawasan tentang kebutuhan pasar lokal yang lebih spesifik.

In [ ]:
# Visualisasi boxplot harga
plt.figure(figsize=(12, 6))
sns.boxplot(x=df_copy['price'])
plt.title('Sebaran Harga di Listings Airbnb')
plt.show()

In [ ]:
# Ambil 5 neighborhood dengan proporsi tertinggi untuk setiap room_type
top5_neigh = {}

for room in crosstab.columns:
    top5_neigh[room] = crosstab[room].sort_values(ascending=False).head(3) * 100  # jadi persen

# Tampilkan hasilnya
for room, top5 in top5_neigh.items():
    print(f"\nTop 5 Neighborhood untuk {room}:")
    print(top5.round(2))

**Insight :**
- Entire home/apt yang mendominasi, Khlong Toei (72.47%), Huai Khwang (68.98%), Sathon (68.81%).
    -  Ini berarti mayoritas besar host di wilayah tersebut menawarkan properti full-unit, yang biasanya relevan untuk target pasar keluarga atau group traveler.
- Entire home/apt yang mendominasi, Phra Nakhon (12.44%), Parthum Wan (9.91%), Lat Krabang (9.40%).
    - Nilainya jauh lebih kecil karena supply hotel room di Airbnb Bangkok secara umum memang terbatas. Proporsi < 15% berarti hotel room di Airbnb bukan pilihan dominan di area mana pun.
- Private room yang mendominasi, Nong Khaem (77.78%), Dusit (77.78%), Don Mueang (68.26%).
    - Angka tinggi ini menunjukkan dominasi listing kamar pribadi di rumah/apartemen yang sudah dihuni, biasanya dari host individu.
- Shared room yang mendominasi, Phra Nakhon (21.16%), Samphanthawong (11.40%), Lat Krabang (10.74%).
    - Angka ini relatif kecil karena supply shared room di Bangkok memang sangat rendah di seluruh wilayah, jadi 21% pun sebenarnya berasal dari total listing yang sedikit.


- Host yang ingin bermain di segmen entire home/apt sebaiknya mempertimbangkan Khlong Toei, Huai Khwang, dan Sathon karena pasarnya besar, tapi perlu analisis lanjut soal persaingan dan harga.
- Untuk private room, wilayah seperti Nong Khaem dan Dusit bisa menjadi niche market bagi traveler budget atau long-term stay.
- Hotel room & shared room sebaiknya hanya jadi opsi jika target pasar benar-benar sesuai, karena demand dan supply di Airbnb Bangkok relatif rendah.

Dengan memahami preferensi ruang dan lokasi, kini saatnya menganalisis faktor lain yang turut mempengaruhi keberhasilan listing: durasi penginapan minimum. Menganalisis sebaran minimum_nights akan membantu memahami pola penginapan di berbagai tipe kamar.

### 1.4 Bagaimana distribusi `minimum_nights` di tiap `room_type`?
Distribusi jumlah `minimum_nights` yang ditetapkan untuk masing-masing tipe kamar memberikan gambaran tentang kebijakan penginapan jangka panjang dan pendek di Airbnb. Memahami distribusi ini penting bagi host untuk menyesuaikan strategi pengelolaan properti, baik untuk pasar yang lebih fleksibel maupun yang lebih terbatas.

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(
    data=df,
    x='room_type',
    y='minimum_nights',
    color='#FF5A5F'  # Airbnb Rausch
)
plt.yscale('log')  # Skala log agar data ekstrem tetap terbaca
plt.title('Distribusi Minimum Nights per Room Type', fontsize=14)
plt.xlabel('Room Type')
plt.ylabel('Minimum Nights (log scale)')
plt.tight_layout()
plt.show()

**Insight :**
- Entire home/apt & Private room punya sebaran lebih lebar
    - Ada banyak outlier ke arah minimum_nights yang tinggi, bahkan ratusan malam.
    - Ini masuk akal kalau ada host yang menetapkan sewa jangka panjang.
- Hotel room & Shared room terlihat seperti flat line
    - Median, Q1, Q3 semuanya di 1 malam, hampir tanpa variasi.
    - Ada sedikit outlier, tapi jumlahnya kecil sekali.
    - Secara bisnis, ini masuk akal kalau hotel & shared room memang cenderung mengizinkan minimal 1 malam, tapi kita perlu pastikan dari data apakah benar hampir semua nilainya 1.
- Potensi masalah data untuk Hotel room & Shared room
    - Kalau nilainya seragam banget di 1 malam, kita perlu cek apakah itu karena memang pola pasar atau karena data entry bias (misalnya scraping hanya menangkap listing dengan nilai default).

In [ ]:
for room in ['Hotel room', 'Shared room']:
    print(f'\nDistribusi minimum_nights untuk {room} (ascending, top 5):')
    print(
        df.query("room_type == @room")['minimum_nights']
          .value_counts(normalize=True)
          .sort_index()
          .head(5)
          .mul(100)
          .round(3)
          .reset_index(name='percent')
          .assign(cumulative_percent=lambda x: x['percent'].cumsum().round(3))
          .to_string(index=False)
    )

**Insight :**
- Lebih dari 91% listing di kedua kategori ini punya minimum_nights = 1.
- Hampir semua sisanya ada di rentang 2–5 malam, sehingga cumulative_percent sudah >96% hanya dari lima nilai terendah.
- Artinya variabilitasnya sangat kecil, jadi boxplot tidak punya “kotak” yang terlihat—semuanya menumpuk di satu titik.
- Jadi ini bukan karena error atau cleaning belum sempurna, tapi memang kondisi data aslinya seperti itu.
Itu kenapa visual boxplot untuk dua kategori ini terlihat rata dan tidak informatif.

Selain itu, kita juga perlu memahami siapa yang menjadi penyedia listing di tiap neighborhood, yaitu tipe host yang terlibat. Analisis distribusi host_type di setiap neighborhood akan melengkapi pemahaman kita tentang siapa yang mendominasi pasar di setiap area.

### 1.5 Bagaimana persebaran `host_type` disetiap neighborhood?
Distribusi tipe host di setiap neighborhood memberi gambaran tentang siapa yang mengelola listing di kawasan tersebut, apakah individu, operator kecil, atau host profesional. Ini memberikan insight penting bagi strategi pemasaran dan optimisasi listing.

In [ ]:
# Buat crosstab untuk melihat proporsi host_type per neighbourhood
crosstab = pd.crosstab(df['neighbourhood'], df['host_type'], normalize='index') * 100

# Urutkan neighbourhood berdasarkan jumlah listing terbanyak
neighbourhood_order = df['neighbourhood'].value_counts().index

# Plot heatmap
plt.figure(figsize=(10, 12))
sns.heatmap(
    crosstab.loc[neighbourhood_order],
    cmap='Reds',
    annot=True,
    fmt='.2f',
    linewidths=.5
)

plt.title('Proporsi Host Type di Setiap Neighborhood di Bangkok', fontsize=14)
plt.xlabel('Host Type')
plt.ylabel('Neighborhood')
plt.tight_layout()
plt.show()

In [ ]:
# ---- TABEL TOP 5 UNTUK SETIAP HOST TYPE ----
for host in df['host_type'].unique():
    top5 = crosstab[host].sort_values(ascending=False).head(5).reset_index()
    top5.columns = ['neighbourhood', 'percent']
    top5['percent'] = top5['percent'].round(2)
    
    print(f"\nTop 5 Neighborhood untuk {host}:")
    print(top5.to_string(index=False))

**Insight :**
- Small Operator paling terkonsentrasi di pinggiran kota atau area dekat bandara, seperti Lat Krabang (48.32%) dan Don Mueang (47.90%), yang keduanya merupakan lokasi strategis untuk wisatawan transit atau perjalanan singkat.
- Single Host sangat dominan di area pinggiran atau permukiman lokal seperti Bang Khun Thain (80.95%), Bang Bon (80.00%), dan Sai Mai (72.50%), menunjukkan bahwa listing di wilayah ini kemungkinan besar dikelola oleh individu atau keluarga, bukan bisnis.
- Professional Host terkonsentrasi di pusat kota atau area wisata utama seperti Sathon (64.85%), Ratchathewi (61.40%), dan Khlong Toei (59.11%), yang masuk akal karena potensi okupansi dan harga lebih tinggi di lokasi premium.

Dengan berbagai analisis ini, maka telah dipahami bagaimana tipe kamar, lokasi, kebijakan penginapan, dan pengelolaan listing berhubungan satu sama lain. Di bagian berikutnya, akan memperdalam pemahaman dengan menggali faktor-faktor lain yang berpotensi memengaruhi performa listing.

## 2. Harga vs okupansi
Harga merupakan salah satu faktor utama yang memengaruhi tingkat okupansi suatu listing di Airbnb. Dalam konteks ini, akan dilakukan analisis untuk mengetahui sejauh mana harga berhubungan dengan `availability_365`, apakah ada harga tertentu yang mencuptakan okupansi optimal, serta bagaimana harga berehubungan dengan `estimated_revenue`.

### 2.1. Apakah harga listing memengaruhi `availability_365`?
Sebelum menyelidiki hubungan antara harga dan okupansi, perlu untuk memahami apakah harga suatu listing memengaruhi seberapa sering listing tersebut tersedia sepanjang tahun. `availability_365` adalah metrik yang menunjukkan berapa hari dalam setahun listing tersebut tersedia untuk pemesanan. Sebuah listing dengan harga tinggi mungkin memiliki tingkat pemesanan yang lebih rendah atau bahkan lebih banyak hari yang diblokir oleh host, sementara harga lebih rendah mungkin membuat listing lebih sering tersedia.

In [ ]:
# Pembagian harga menjadi dua kategori: Low dan High berdasarkan median
median_price = df['price'].median()
df['price_category'] = df['price'].apply(lambda x: 'Low' if x <= median_price else 'High')

In [ ]:
# Siapkan data untuk t-test (availability_365 untuk harga rendah dan tinggi)
low_price = df[df['price_category'] == 'Low']['availability_365']
high_price = df[df['price_category'] == 'High']['availability_365']

In [ ]:
sns.histplot(low_price)

In [ ]:
# Membagi data harga menjadi dua kategori: Low dan High berdasarkan median
median_price = df['price'].median()
df['price_category'] = df['price'].apply(lambda x: 'Low' if x <= median_price else 'High')

# Menyiapkan data untuk T-test (availability_365 untuk harga rendah dan tinggi)
low_price = df[df['price_category'] == 'Low']['availability_365']
high_price = df[df['price_category'] == 'High']['availability_365']

# Independent t-test untuk perbedaan antara harga rendah dan tinggi dalam hal availability_365
t_stat, p_value = stats.ttest_ind(low_price, high_price)

# Menampilkan hasil uji t-test
print(f"T-Statistik: {t_stat:.2f}")
print(f"P-Value: {p_value:.2f}")

# Interpretasi hasil t-test
if p_value > 0.05:
    print(f'p_value = {p_value:.2f}, Terima H0: Tidak ada perbedaan signifikan antara harga Low dan High.')
else:
    print(f'p_value = {p_value:.2f}, Terima H1: Ada perbedaan signifikan antara harga Low dan High.')

### 2.2 Adakah 'sweet spot' harga dengan okupansi optimal?
Banyak host QIrbnb mungkin bertanya-tanya, "Berapa harga yang sebaiknya saya tetapkan untuk mendapatkan okupansi optimal?" Dalam analisis ini, akan dicari tahu apakah ada harga tertentu atau yang sering disebut sebagai 'sweet spot' yang menghasilkan tingkat okupansi tertinggi. Indentifikasi ini sangat berguna untuk menentukan harga yang seimbang antara permintaan pasar dan potensi pendapatan.

### 2.3 Bagaimana distribusi `estimated_reveneu` di tiap `price_category`?
Selain okupansi, salah satu faktor penting yang perlu dipertimbangkan adalah `estimated_revenue` atau pendapatan yang diperkirakan dari suatu listing berdasarkan harga dan okupansi. Memahami bagaimana pendapatan ini tersebar di berbagai `price_category` dapat memberikan wawasan lebih lanjut tentang seberapa efisien setiap kategori harga dalam menghasilkan pendapatan bagi host.

### 2.4 Apakah `room_type` memengaruhi hubungan price dan okupansi?
`room_type` juga berpotensi memengaruhi bagaimana harga dan okupansi saling berhubungan. Misalnya, harga untuk entire home/apt mungkin lebih tinggi daripada private room, tetapi okupansi untuk tipe kamar yang lebih terjangkau mungkin lebih tinggi. Oleh karena itu, penting untuk mengeksplorasi apakah `room_type` memengaruhi hubungan antara harga dan okupansi, dan apakah ada perbedaan yang signifikan antara tipe kamar dalam hal ini.

### 2.5 Apakah `profesional_host` lebih sering mengatur harga di segmen premium?
Host profesional seringkali memiliki lebih banyak pengalaman dan sumber daya untuk menetapkan harga yang sesuai dengan pasar. Oleh karena itu, perlu diketahui apakah profesional host lebih cenderung untuk menetapkan harga di segmen premium (harga tinggi) dibandingkan dengan single host atau host non profesional. Memahami pervedaan ini bisa memberikan wawasan tentang strategi harga yang ditetapkan oleh profesional.

## 3. Karakteristik listing populer (number_of_reviews_ltm)

### 3.1 Tipe room apa yang paling banyak direview?

### 3.2 Neighborhood mana yang listingnya paling populer?

### 3.3 Bagaimana hubungan `price_category` dengan jumlah review?

### 3.4 Apakah `host_type` tertentu lebih sering mendapatkan review tinggi?

### 3.5 Seberapa aktif listing populer diukur dari `review_recency`?

## 4. Performa host profesional vs non profesional

### 4.1 Apakah `professional_host` memiliki availability yang lebih tinggi?

### 4.2 Apakah `estimated_revenue` `professional_host` lebih tinggi dibandingkan `single_host`?

### 4.3 Bagaimana distribusi `price_category` antara `professional_host` dan `single_host`?

### 4.4 Apakah `professional_host` lebih banyak listing di neighborhood premium?

### 4.5 Bagaimana performa engagement (reviews) antara `professional_host` dan lainnya?

- Pengaruh recency reviews terhadap performa listing
- Sub-questions:
    - Bagaimana ditribusi listing dalam kategori review_recency?
    - Apakah listing dengan review terbaru memiliki price lebih tinggi?
    - Apakah estimated_occupancy_rate listing aktif lebih tinggi?
    - Seberapa besar gap performa revenue antara listing aktif vs pasif?
    - Apakah host_type berpengaruh terhadap recency_review?
- Kolom Data: review_recency, price, estimated_occupancy_rate, estimated_revenue, host_type